# task 3 : 打标签，生成原始数据集

In [2]:
import os,json,sys,logging
sys.path.append("./share")
sys.path.append("./common")
import pandas as pd
import numpy as np
from IoTCommon import CIoTCommon
from Config import g_data_root
from tqdm.notebook import tqdm
import warnings,os,json,csv
warnings.simplefilter("ignore")
tqdm.pandas()

In [4]:
g_attack_ip = {
    "Backdoor_attack":["192.168.0.170"],
    "Password attacks":["192.168.0.170"],
    "Ransomware attack":["192.168.0.170"],
    "Uploading attack":["192.168.0.170"],
    "SQL injection attack":["192.168.0.170"],
    "XSS attacks":["192.168.0.170","172.217.19.42","104.16.87.20"],
    "MITM (ARP spoofing + DNS) Attack":["192.168.0.101","192.168.0.152","172.217.19.35","192.168.0.170"],
    "Vulnerability scanner attack":["192.168.0.170","142.250.200.205","172.217.19.35","142.250.201.10"],
    "OS Fingerprinting attack":["192.168.0.170"],
    "Port Scanning attack":["192.168.0.170"],
    "DDoS ICMP Flood Attacks":["213.117.18.213","183.223.100.122","166.153.227.121","49.81.59.152","227.117.33.125"],
    "DDoS HTTP Flood Attacks":["192.168.0.170","216.58.198.74"],
    "DDoS UDP Flood Attacks":["190.123.219.128","16.226.184.201","153.125.214.15","91.184.12.91"],
    "DDoS TCP SYN Flood Attacks":["207.192.25.133","94.196.109.185","133.149.252.77","220.146.94.148"]
}
g_data_set = g_data_root + "dataset/"

In [3]:
for fi in tqdm(CIoTCommon.get_json_files("%spcap-json/"%g_data_root)):
    fi = fi.replace("\\","/")
    Category,AttackType,file_name = fi.split("/")[-3],fi.split("/")[-2],fi.split("/")[-1]
    directory = os.path.dirname(fi)
    directory = directory.replace("%s"%g_data_root,g_data_set).replace("/pcap-json","")
    new_file_name = "%s/%s"%(directory, file_name)
    #new_file_name = new_file_name.replace(".json",".csv")
    
    if Category == "Attack":
        ip_list = g_attack_ip[AttackType]
    else:
        ip_list = []        
    data = CIoTCommon.get_feature_data(fi)
    if data:
        df_tmp = pd.DataFrame(data)
        withIp = False
        
        if 'ip.src' in df_tmp.keys().tolist():
            mask = df_tmp['ip.src'].isin(ip_list) | df_tmp['ip.dst'].isin(ip_list)
            withIp = True
            
        if 'arp.src.proto_ipv4' in df_tmp.keys().tolist():
            if not withIp:
                mask = df_tmp['arp.src.proto_ipv4'].isin(ip_list) | df_tmp['arp.dst.proto_ipv4'].isin(ip_list)
            else:
                mask = mask | df_tmp['arp.src.proto_ipv4'].isin(ip_list) | df_tmp['arp.dst.proto_ipv4'].isin(ip_list)
        
        df_tmp.loc[mask,"Label"] = '1'
        df_tmp.loc[~mask,"Label"] = '0'
        df_tmp["Label"].astype(int)
        
        os.makedirs(directory, exist_ok=True)
        df_tmp.to_json(new_file_name,orient='records')
        #df_tmp.to_csv(new_file_name,index=False,sep=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, doublequote=True, encoding='utf-8')

0it [00:00, ?it/s]